In [2]:
import pandas as pd
import numpy as np
train_data=pd.read_csv("	rain_F3WbcTw.csv")

In [3]:
print (train_data.shape)
train_data.head()

(5279, 4)


,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [4]:
print (train_data.shape)
print (train_data.isnull().sum())
print (f"{train_data.unique_hash.nunique()} unique hash") #all unique_hash values are unique 
print (f"{train_data.drug.nunique()}  unique drugs")        #we have 102 unique drugs
print (train_data.sentiment.value_counts()) #3 sentiments

(5279, 4)
unique_hash    0
text           0
drug           0
sentiment      0
dtype: int64
5279 unique hash
102  unique drugs
2    3825
1     837
0     617
Name: sentiment, dtype: int64


In [570]:
print (test_data.shape)
print (test_data.isnull().sum())
print (test_data.unique_hash.nunique()) #all unique_hash values are unique 
print (test_data.drug.nunique())        #we have 102 unique drugs

(2924, 3)
unique_hash    0
text           0
drug           0
dtype: int64
2924
95


In [5]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
train_data["drug"]=label.fit_transform(train_data["drug"])
train_data.drop("unique_hash",axis=1,inplace=True)
train_data.head()

,text,drug,sentiment
0,Autoimmune diseases tend to come in clusters. ...,38,2
1,I can completely understand why you’d want to ...,38,2
2,Interesting that it only targets S1P-1/5 recep...,35,2
3,"Very interesting, grand merci. Now I wonder wh...",64,2
4,"Hi everybody, My latest MRI results for Brain ...",38,1


In [6]:
"""
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,PorterStemmer
port=PorterStemmer()
word_=WordNetLemmatizer()
def lemmatize_stemming(text):
    return port.stem(word_.lemmatize(text, pos='v'))
def preprocess(text):
    result=[]
    print (text)
    for token in simple_preprocess(text):
        print (token)
        if token not in STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
    joined_words=" ".join(result)
    return joined_words
"""

from nltk.corpus import stopwords
sw = stopwords.words("english")
import re
from nltk.stem.snowball import SnowballStemmer,PorterStemmer
import string
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    ## Remove stop words
    text = [w for w in text if not w in sw and len(w) >= 3]
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    txt = text.split()
    text = [i for i in txt if i.isalpha()]
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    text=text.lower()
    return text

In [7]:
#x=train_data["text"][0]
#print (x)
#x=train_data["text"][0].apply(preprocess)
#preprocess(x)
#x
#descp=train_data["text"].apply(preprocess)
descp=train_data["text"].apply(clean_text)

In [8]:
#train_data.to_csv("train_innoplexus.csv")
descp[0]
train_data.head()

,text,drug,sentiment
0,Autoimmune diseases tend to come in clusters. ...,38,2
1,I can completely understand why you’d want to ...,38,2
2,Interesting that it only targets S1P-1/5 recep...,35,2
3,"Very interesting, grand merci. Now I wonder wh...",64,2
4,"Hi everybody, My latest MRI results for Brain ...",38,1


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec=TfidfVectorizer(min_df=3, max_features=1000,strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 4), use_idf=1, smooth_idf=1, sublinear_tf=1)
transvec=tfvec.fit_transform(descp)

In [10]:
from sklearn.decomposition import TruncatedSVD
tsvd=TruncatedSVD(n_components=50)
data_text=tsvd.fit_transform(transvec)
new_data=pd.DataFrame(data_text,columns=["n"+str(i) for i in range(50)])

In [11]:
train_mod=pd.concat([train_data,new_data],axis=1)
train_mod.drop("text",axis=1,inplace=True)
train_mod.head(2)

,drug,sentiment,n0,n1,n2,n3,n4,n5,n6,n7,...,n40,n41,n42,n43,n44,n45,n46,n47,n48,n49
0,38,2,0.292797,-0.210832,0.037319,-0.248554,-0.124324,-0.006031,0.215588,0.198353,...,0.033431,0.073548,0.015384,0.022584,-0.093104,0.02066,-0.016259,0.058313,0.011339,-0.012436
1,38,2,0.363099,-0.084139,-0.037395,-0.184300,-0.155908,0.030592,0.182617,0.005156,...,0.041814,-0.030149,0.049422,0.043905,0.039392,0.01003,0.036370,0.032603,0.057347,0.056890


In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_mod.drop("sentiment",axis=1,inplace=True)
#scaled=train_mod.loc[:,train_mod.columns!="sentiment"]
scaled=sc.fit_transform(train_mod)
print (scaled.shape)
print (type(scaled))

(5279, 51)
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [13]:
train_mod.shape

(5279, 51)

In [14]:
modified_training_data=pd.DataFrame(data=scaled,columns=train_mod.columns)
modified_training_data.head(2)

,drug,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n40,n41,n42,n43,n44,n45,n46,n47,n48,n49
0,-0.648663,0.156338,-1.346728,0.303039,-2.168141,-1.416341,-0.150591,2.372560,2.333787,0.930002,...,0.648471,1.426587,0.287815,0.464816,-1.817205,0.421050,-0.346911,1.183177,0.225349,-0.253613
1,-0.648663,0.830113,-0.526050,-0.297917,-1.607420,-1.751460,0.240289,2.014315,0.049432,-0.343953,...,0.809820,-0.578225,0.952456,0.886292,0.807237,0.209942,0.703559,0.664646,1.158057,1.165381


In [16]:
#train_mod=train_mod.drop(["unique_hash"],axis=1)
#train_mod.drop("unique_hash",axis=1,inplace=True)
from sklearn.utils import shuffle
modified_training_data=shuffle(modified_training_data)

#X=modified_training_data.loc[:,modified_training_data.columns!="sentiment"]
X=modified_training_data
Y=train_data["sentiment"]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [17]:
modified_training_data.head(2)
y_train.head(2)

1738    2
3771    1
Name: sentiment, dtype: int64

In [18]:
y_train.head()

1738    2
3771    1
2634    2
4199    2
2694    0
Name: sentiment, dtype: int64

In [64]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
pred=dt.predict(x_test)
print (accuracy_score(y_test,pred))
f1_score(y_test,pred)

0.5618686868686869


ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=500)
rfc.fit(x_train,y_train)
pred2=rfc.predict(x_test)
accuracy_score(y_test,pred2)

0.7234848484848485

In [21]:
from sklearn.svm import SVC
sv=SVC()
sv.fit(x_train,y_train)
pred3=sv.predict(x_test)
accuracy_score(y_test,pred3)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7266414141414141

In [22]:
from sklearn.linear_model import RidgeClassifier
rc=RidgeClassifier()
rc.fit(x_train,y_train)
pred4=rc.predict(x_test)
accuracy_score(y_test,pred4)

0.7266414141414141

In [23]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier()
sgd.fit(x_train,y_train)
pred5=sgd.predict(x_test)
accuracy_score(y_test,pred5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.5997474747474747

In [24]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(multi_class='multinomial',C=1,solver='saga')
lr.fit(x_train,y_train)
pred6=lr.predict(x_test)
accuracy_score(y_test,pred6)


0.7266414141414141

In [25]:
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate =0.04,n_estimators=2000,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,seed=27)

xgb.fit(x_train,y_train)
pred7 = xgb.predict(x_test)
accuracy_score(y_test,pred7)


0.7178030303030303

In [26]:
from catboost import CatBoostClassifier
cb=CatBoostClassifier()
cb.fit(x_train,y_train)
pred8=cb.predict(x_test)
accuracy_score(y_test,pred8)

0:	learn: -1.0794456	total: 1.04s	remaining: 17m 22s
1:	learn: -1.0616235	total: 1.88s	remaining: 15m 36s
2:	learn: -1.0452161	total: 2.7s	remaining: 14m 56s
3:	learn: -1.0296838	total: 3.5s	remaining: 14m 30s
4:	learn: -1.0147875	total: 4.23s	remaining: 14m 2s
5:	learn: -1.0009376	total: 5.03s	remaining: 13m 53s
6:	learn: -0.9880323	total: 5.83s	remaining: 13m 46s
7:	learn: -0.9759032	total: 6.61s	remaining: 13m 39s
8:	learn: -0.9641873	total: 7.32s	remaining: 13m 25s
9:	learn: -0.9535195	total: 8.11s	remaining: 13m 23s
10:	learn: -0.9434566	total: 8.94s	remaining: 13m 24s
11:	learn: -0.9340283	total: 9.83s	remaining: 13m 29s
12:	learn: -0.9248824	total: 10.6s	remaining: 13m 22s
13:	learn: -0.9161777	total: 11.4s	remaining: 13m 20s
14:	learn: -0.9081275	total: 12.1s	remaining: 13m 16s
15:	learn: -0.9003958	total: 12.9s	remaining: 13m 11s
16:	learn: -0.8931151	total: 13.6s	remaining: 13m 9s
17:	learn: -0.8863696	total: 14.4s	remaining: 13m 4s
18:	learn: -0.8799491	total: 15.1s	remainin

151:	learn: -0.7268699	total: 2m 4s	remaining: 11m 32s
152:	learn: -0.7265965	total: 2m 4s	remaining: 11m 31s
153:	learn: -0.7258230	total: 2m 5s	remaining: 11m 32s
154:	learn: -0.7253673	total: 2m 6s	remaining: 11m 31s
155:	learn: -0.7251759	total: 2m 7s	remaining: 11m 30s
156:	learn: -0.7249079	total: 2m 8s	remaining: 11m 29s
157:	learn: -0.7244387	total: 2m 9s	remaining: 11m 27s
158:	learn: -0.7239337	total: 2m 9s	remaining: 11m 27s
159:	learn: -0.7234928	total: 2m 10s	remaining: 11m 26s
160:	learn: -0.7232288	total: 2m 11s	remaining: 11m 24s
161:	learn: -0.7223201	total: 2m 12s	remaining: 11m 24s
162:	learn: -0.7218534	total: 2m 12s	remaining: 11m 22s
163:	learn: -0.7215413	total: 2m 13s	remaining: 11m 21s
164:	learn: -0.7210852	total: 2m 14s	remaining: 11m 20s
165:	learn: -0.7203074	total: 2m 15s	remaining: 11m 19s
166:	learn: -0.7199913	total: 2m 16s	remaining: 11m 18s
167:	learn: -0.7195063	total: 2m 16s	remaining: 11m 17s
168:	learn: -0.7190454	total: 2m 17s	remaining: 11m 16s


300:	learn: -0.6781125	total: 4m	remaining: 9m 18s
301:	learn: -0.6778166	total: 4m 1s	remaining: 9m 17s
302:	learn: -0.6774763	total: 4m 1s	remaining: 9m 16s
303:	learn: -0.6771925	total: 4m 2s	remaining: 9m 15s
304:	learn: -0.6768237	total: 4m 3s	remaining: 9m 14s
305:	learn: -0.6765559	total: 4m 4s	remaining: 9m 13s
306:	learn: -0.6763847	total: 4m 4s	remaining: 9m 12s
307:	learn: -0.6761619	total: 4m 5s	remaining: 9m 11s
308:	learn: -0.6758830	total: 4m 6s	remaining: 9m 10s
309:	learn: -0.6755749	total: 4m 7s	remaining: 9m 9s
310:	learn: -0.6751791	total: 4m 7s	remaining: 9m 9s
311:	learn: -0.6750297	total: 4m 8s	remaining: 9m 8s
312:	learn: -0.6747846	total: 4m 9s	remaining: 9m 7s
313:	learn: -0.6746347	total: 4m 9s	remaining: 9m 6s
314:	learn: -0.6743808	total: 4m 10s	remaining: 9m 5s
315:	learn: -0.6741294	total: 4m 11s	remaining: 9m 4s
316:	learn: -0.6738665	total: 4m 12s	remaining: 9m 3s
317:	learn: -0.6735620	total: 4m 13s	remaining: 9m 2s
318:	learn: -0.6732737	total: 4m 13s

451:	learn: -0.6338896	total: 5m 54s	remaining: 7m 10s
452:	learn: -0.6336256	total: 5m 55s	remaining: 7m 9s
453:	learn: -0.6332804	total: 5m 56s	remaining: 7m 8s
454:	learn: -0.6330085	total: 5m 56s	remaining: 7m 7s
455:	learn: -0.6326395	total: 5m 57s	remaining: 7m 6s
456:	learn: -0.6323304	total: 5m 58s	remaining: 7m 6s
457:	learn: -0.6320875	total: 5m 59s	remaining: 7m 5s
458:	learn: -0.6316599	total: 6m	remaining: 7m 4s
459:	learn: -0.6314393	total: 6m	remaining: 7m 3s
460:	learn: -0.6311815	total: 6m 1s	remaining: 7m 2s
461:	learn: -0.6308661	total: 6m 2s	remaining: 7m 2s
462:	learn: -0.6306996	total: 6m 3s	remaining: 7m 1s
463:	learn: -0.6304800	total: 6m 4s	remaining: 7m
464:	learn: -0.6302869	total: 6m 4s	remaining: 6m 59s
465:	learn: -0.6300134	total: 6m 5s	remaining: 6m 58s
466:	learn: -0.6295642	total: 6m 6s	remaining: 6m 57s
467:	learn: -0.6292115	total: 6m 6s	remaining: 6m 57s
468:	learn: -0.6289792	total: 6m 7s	remaining: 6m 56s
469:	learn: -0.6286304	total: 6m 8s	remain

602:	learn: -0.5935462	total: 7m 50s	remaining: 5m 9s
603:	learn: -0.5933418	total: 7m 51s	remaining: 5m 9s
604:	learn: -0.5931733	total: 7m 52s	remaining: 5m 8s
605:	learn: -0.5930420	total: 7m 52s	remaining: 5m 7s
606:	learn: -0.5928002	total: 7m 53s	remaining: 5m 6s
607:	learn: -0.5925573	total: 7m 54s	remaining: 5m 5s
608:	learn: -0.5923912	total: 7m 55s	remaining: 5m 5s
609:	learn: -0.5919459	total: 7m 56s	remaining: 5m 4s
610:	learn: -0.5916940	total: 7m 56s	remaining: 5m 3s
611:	learn: -0.5914604	total: 7m 57s	remaining: 5m 2s
612:	learn: -0.5911016	total: 7m 58s	remaining: 5m 1s
613:	learn: -0.5910088	total: 7m 58s	remaining: 5m 1s
614:	learn: -0.5908223	total: 7m 59s	remaining: 5m
615:	learn: -0.5904554	total: 8m	remaining: 4m 59s
616:	learn: -0.5901429	total: 8m 1s	remaining: 4m 58s
617:	learn: -0.5899240	total: 8m 1s	remaining: 4m 57s
618:	learn: -0.5896723	total: 8m 2s	remaining: 4m 57s
619:	learn: -0.5894237	total: 8m 3s	remaining: 4m 56s
620:	learn: -0.5892185	total: 8m 4

753:	learn: -0.5588255	total: 9m 15s	remaining: 3m 1s
754:	learn: -0.5587085	total: 9m 15s	remaining: 3m
755:	learn: -0.5585457	total: 9m 16s	remaining: 2m 59s
756:	learn: -0.5582185	total: 9m 16s	remaining: 2m 58s
757:	learn: -0.5580793	total: 9m 17s	remaining: 2m 57s
758:	learn: -0.5579070	total: 9m 17s	remaining: 2m 56s
759:	learn: -0.5576614	total: 9m 17s	remaining: 2m 56s
760:	learn: -0.5575167	total: 9m 18s	remaining: 2m 55s
761:	learn: -0.5572865	total: 9m 18s	remaining: 2m 54s
762:	learn: -0.5569624	total: 9m 19s	remaining: 2m 53s
763:	learn: -0.5568412	total: 9m 19s	remaining: 2m 52s
764:	learn: -0.5564913	total: 9m 19s	remaining: 2m 52s
765:	learn: -0.5563241	total: 9m 20s	remaining: 2m 51s
766:	learn: -0.5560219	total: 9m 20s	remaining: 2m 50s
767:	learn: -0.5557174	total: 9m 21s	remaining: 2m 49s
768:	learn: -0.5554848	total: 9m 21s	remaining: 2m 48s
769:	learn: -0.5553950	total: 9m 22s	remaining: 2m 47s
770:	learn: -0.5550680	total: 9m 22s	remaining: 2m 47s
771:	learn: -0.

903:	learn: -0.5266256	total: 10m 19s	remaining: 1m 5s
904:	learn: -0.5265061	total: 10m 19s	remaining: 1m 5s
905:	learn: -0.5263441	total: 10m 20s	remaining: 1m 4s
906:	learn: -0.5261828	total: 10m 20s	remaining: 1m 3s
907:	learn: -0.5260954	total: 10m 21s	remaining: 1m 2s
908:	learn: -0.5259209	total: 10m 21s	remaining: 1m 2s
909:	learn: -0.5257497	total: 10m 21s	remaining: 1m 1s
910:	learn: -0.5256327	total: 10m 22s	remaining: 1m
911:	learn: -0.5253790	total: 10m 22s	remaining: 1m
912:	learn: -0.5252187	total: 10m 23s	remaining: 59.4s
913:	learn: -0.5250868	total: 10m 23s	remaining: 58.7s
914:	learn: -0.5249186	total: 10m 23s	remaining: 58s
915:	learn: -0.5246802	total: 10m 24s	remaining: 57.3s
916:	learn: -0.5245305	total: 10m 24s	remaining: 56.6s
917:	learn: -0.5242471	total: 10m 25s	remaining: 55.8s
918:	learn: -0.5238605	total: 10m 25s	remaining: 55.1s
919:	learn: -0.5238041	total: 10m 26s	remaining: 54.4s
920:	learn: -0.5236185	total: 10m 26s	remaining: 53.7s
921:	learn: -0.523

0.7266414141414141

In [ ]:
###Operation with Test data

In [53]:
test_data=pd.read_csv("	est_tOlRoBf.csv")
test_data2=pd.read_csv("	est_tOlRoBf.csv")
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
#test_data["unique_hash"]=label.fit_transform(test_data["unique_hash"])
test_data["drug"]=label.fit_transform(test_data["drug"])
test_data.drop("unique_hash",axis=1,inplace=True)
test_data.head()

,text,drug
0,"256 (previously stable on natalizumab), with 5...",33
1,On fingolimod and have been since December 201...,33
2,Apparently it's shingles! :-/ I do have a few ...,39
3,If the Docetaxel doing once a week x3 weeks th...,81
4,"CC, Stelara worked in a matter of days for me....",78


In [54]:
test_text=test_data["text"].apply(clean_text)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec_test=TfidfVectorizer(min_df=3, max_features=1000,strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 4), use_idf=1, smooth_idf=1, sublinear_tf=1)
transvec_test=tfvec_test.fit_transform(test_text)

In [56]:
x_train.shape

(3695, 51)

In [57]:
tsvd_test=TruncatedSVD(n_components=50)
test_data_text=tsvd_test.fit_transform(transvec_test)
test_new_data=pd.DataFrame(test_data_text,columns=["n"+str(i) for i in range(50)])

In [58]:
test_mod=pd.concat([test_data,test_new_data],axis=1)
test_mod.drop("text",axis=1,inplace=True)
test_mod.head(2)

,drug,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n40,n41,n42,n43,n44,n45,n46,n47,n48,n49
0,33,0.053811,0.024212,-0.033774,-0.031532,-0.055755,-0.002924,0.134186,0.052732,0.041644,...,0.130018,0.149075,-0.008368,0.034003,0.039727,0.064267,0.099432,-0.111729,-0.011905,-0.049151
1,33,0.124385,-0.046962,-0.005413,-0.062577,-0.008021,0.029231,0.018559,0.007530,-0.005441,...,0.177769,0.032538,-0.096181,-0.043586,-0.022599,-0.088677,0.096256,-0.026726,0.001448,-0.037403


In [59]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
test_mod=sc.fit_transform(test_mod)
test_mod.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(2924, 51)

In [70]:
print (test_mod.shape)
test_pred=sv.predict(test_mod)
submission=pd.DataFrame(test_pred)
#submission.insert(0,'unique_hash',test_data["unique_hash"])
submission.columns=["sentiment"]
submission.insert(0,'unique_hash',test_data2['unique_hash'])
submission=submission.round(0)
submission.head()

(2924, 51)


,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,2
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,2
2,50b6d851bcff4f35afe354937949e9948975adf7,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,2


In [71]:
submission.to_csv("sample_submission.csv",index=False)

In [72]:
submission['sentiment'].value_counts()

2    2924
Name: sentiment, dtype: int64